# Multi message generator
This generator example can be used to send multiple Kafka messages at once. The messages can be routed to multiple Kafka topics and can be different protobuf structures. It also alows you to put delay between messages.

**Note: You do not have to use this notebook. You can finish the lab with the simple generator cells. This notebook illustrates a possible solution to create some more elaborate use cases / automatization.**

## How to use
- Import the needed protobuf structures.
- Create a function such as `create_user_rating` where you create the protobuf message. The function **must** return a `GeneratedMessage` tuple. This tuple has 3 fields: topic, key and message. `topic` is the Kafka topic where the message has to be routed to. `key` is the Kafka key of the message and **must** be serialized. The `message` is your (unserialized) protobuf message.
- Add your function to the `function_calls` list. Each item in this list is a `CallableFunction` tuple. This tuple has 2 fields. The first `function` is a reference to your function and the second `args` is a list of arguments for your function. The `function_calls` list will execute the items one by one. To create delay between messages you can use `sleep`. 


In [ ]:
from collections import namedtuple
from time import sleep

from kafka import KafkaProducer
# Import your protobuf structures here
from messages_pb2 import UserRating, UserRecommendedMoviesRequest

CallableFunction = namedtuple('CallableFunction', ['function', 'args'])
GeneratedMessage = namedtuple('GeneratedMessage', ['topic', 'key', 'message'])


# Write functions to create a protobuf structures here.
# Each function should return a GeneratedMessage tuple.
# A GeneratedMessage has a topic, key and message field.
def create_user_recommended_movies_request(user_id):
    urmr = UserRecommendedMoviesRequest()
    urmr.user_id = int(user_id)
    return GeneratedMessage('userrecommovies', repr(user_id).encode('utf-8'), urmr)


def create_user_rating(user_id, movie_id, rating, timestamp):
    ur = UserRating()
    ur.userid = user_id
    ur.movieid = movie_id
    ur.rating = rating
    ur.timestamp = timestamp
    return GeneratedMessage('userratings', repr(user_id).encode('utf-8'), ur)


# Add all function calls here in a CallableFunction tuple.
# CallableFunction takes a function reference and a list with function arguments.
function_calls = [
        CallableFunction(create_user_rating, [603, 256, 3.0, 1514783105000]),
        CallableFunction(sleep, [5]), # Wait 5 seconds before sending the next request
        CallableFunction(create_user_recommended_movies_request, [603]),
    ]

######################################################
# You do not have to change anything below this line #
######################################################

def main():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
    
    for f in function_calls:
        message = wrapper(f.function, f.args)
        if message: # If the functions was a sleep we don't have to send anything
            producer.send(
                topic=message.topic,
                key=message.key,
                value=message.SerializeToString()
            )
            producer.flush()


def wrapper(func, args):
    return func(*args)


if __name__ == "__main__":
    main()